In [1]:
import json
import random
from random import shuffle
from pathlib import Path
from tqdm import tqdm
import numpy as np
import pandas as pd
import torch
from torch_scatter import scatter
from torch_geometric.data import Data, InMemoryDataset
from torch_geometric.data import Dataset
from feature import one_of_k_encoding, toxcast_tasks
from utils import random_scaffold_split

import rdkit
from rdkit import Chem, RDConfig
from rdkit.Chem import ChemicalFeatures, MolFromSmiles
from rdkit.Chem.rdchem import HybridizationType as HT
from rdkit.Chem.rdchem import BondType as BT
from rdkit import RDLogger

from model import *
import torch
import pandas as pd
from torch.utils.data import DataLoader
from dataset import Dataset

from glam import *

In [2]:
model = Architecture()

In [3]:
checkpoint = torch.load("./log_bbbp/2024-04-12_05:22:24.365_seed_12/best_save.ckpt")

In [4]:
model.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

In [5]:
model.eval()

Architecture(
  (mol_lin0): LinearBlock(
    (norm): _None()
    (dropout): _None()
    (linear): Linear(in_features=15, out_features=60, bias=True)
    (act): RReLU(lower=0.125, upper=0.3333333333333333)
  )
  (mol_conv): MessageBlock(
    (norm): _None()
    (dropout): Dropout(p=0.2, inplace=False)
    (conv): _NNConv(
      (conv): NNConv(60, 60, aggr=mean, nn=Sequential(
        (0): Linear(in_features=4, out_features=32, bias=True)
        (1): ReLU()
        (2): Linear(in_features=32, out_features=3600, bias=True)
      ))
    )
    (gru): GRU(60, 60)
    (act): RReLU(lower=0.125, upper=0.3333333333333333)
  )
  (mol_readout): GlobalPool5()
  (mol_flat): LinearBlock(
    (norm): _None()
    (dropout): _None()
    (linear): Linear(in_features=300, out_features=1024, bias=True)
    (act): RReLU(lower=0.125, upper=0.3333333333333333)
  )
  (lin_out1): LinearBlock(
    (norm): _None()
    (dropout): Dropout(p=0.2, inplace=False)
    (linear): Linear(in_features=1024, out_features=1,

In [6]:
#dataset = Dataset(root="./main_test/", dataset="tanimoto",)
#dataset.process()

In [7]:
mol_batch = torch.load('./main_test/processed/dataset_tanimoto.pt')

In [8]:
mol_batch

(Data(x=[62044, 15], edge_index=[2, 134982], edge_attr=[134982, 4], y=[2000, 1], smi=[2000]),
 {'x': tensor([    0,    33,    66,  ..., 61984, 62014, 62044]),
  'edge_index': tensor([     0,     72,    148,  ..., 134854, 134918, 134982]),
  'edge_attr': tensor([     0,     72,    148,  ..., 134854, 134918, 134982]),
  'y': tensor([   0,    1,    2,  ..., 1998, 1999, 2000]),
  'smi': tensor([   0,    1,    2,  ..., 1998, 1999, 2000])})

In [9]:
mol_batch_0 = mol_batch[0]

In [16]:
mol_batch_0

Data(x=[62044, 15], edge_index=[2, 134982], edge_attr=[134982, 4], y=[2000, 1], smi=[2000])

In [70]:
mol_batch_0["x"][52344]

tensor([0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 6., 0., 0.])

In [10]:
mol_batch_1 = mol_batch[1]

In [58]:
mol_batch_1

{'x': tensor([    0,    33,    66,  ..., 61984, 62014, 62044]),
 'edge_index': tensor([     0,     72,    148,  ..., 134854, 134918, 134982]),
 'edge_attr': tensor([     0,     72,    148,  ..., 134854, 134918, 134982]),
 'y': tensor([   0,    1,    2,  ..., 1998, 1999, 2000]),
 'smi': tensor([   0,    1,    2,  ..., 1998, 1999, 2000])}

In [64]:
mol_batch_0

Data(x=[62044, 15], edge_index=[2, 134982], edge_attr=[134982, 4], y=[2000, 1], smi=[2000])

In [65]:
mol_batch_0["x"]

tensor([[0., 1., 0.,  ..., 6., 0., 0.],
        [0., 1., 0.,  ..., 6., 1., 0.],
        [0., 1., 0.,  ..., 6., 1., 0.],
        ...,
        [0., 1., 0.,  ..., 6., 1., 0.],
        [0., 1., 0.,  ..., 6., 1., 0.],
        [0., 1., 0.,  ..., 6., 1., 0.]])

In [71]:
mol_batch_iter = iter(mol_batch_0)

In [80]:
next(mol_batch_iter)

StopIteration: 

In [79]:
for i in mol_batch_iter:
    print(i)

In [57]:
result = model(mol_batch_0)

/home/media/data2/user_home/dbsejrgus226/miniconda3/envs/GLAM/lib/python3.11/site-packages/torch_geometric/deprecation.py:26: UserWarning: 'nn.glob.global_sort_pool' is deprecated, use 'nn.aggr.SortAggr' instead
  warnings.warn(out)


In [45]:
result

tensor([[-6762.1475]], grad_fn=<AddmmBackward0>)

In [54]:
mol_batch_x = mol_batch_0["x"]

In [56]:
model(mol_batch_x[0])

AttributeError: 'Tensor' object has no attribute 'x'

In [52]:
predictions = []

for input in mol_batch_x:
    print(input.shape)
    result = model(input)
    predictions.append(result)

torch.Size([15])


AttributeError: 'Tensor' object has no attribute 'x'

In [5]:
import pickle

In [6]:
with open("./output.pk", "rb") as f:
    data = pickle.load(f)

data

[tensor([[0.8597],
         [0.8711],
         [0.8811],
         [0.6916],
         [0.9249],
         [0.8524],
         [0.8367],
         [0.9591],
         [0.8607],
         [0.8524],
         [0.9755],
         [0.8550],
         [0.9356],
         [0.8711],
         [0.9021],
         [0.9371],
         [0.8499],
         [0.8499],
         [0.8866],
         [0.6916],
         [0.7547],
         [0.9609],
         [0.8905],
         [0.9090],
         [0.9299],
         [0.7547],
         [0.9112],
         [0.8866],
         [0.9043],
         [0.9249],
         [0.9637],
         [0.7844]]),
 tensor([[0.9016],
         [0.8711],
         [0.8550],
         [0.9258],
         [0.9005],
         [0.8491],
         [0.8553],
         [0.9605],
         [0.8774],
         [0.8959],
         [0.8905],
         [0.7844],
         [0.7990],
         [0.9258],
         [0.8711],
         [0.9755],
         [0.8885],
         [0.8985],
         [0.8885],
         [0.9605],
         [

In [7]:
len(data)

63

In [8]:
with open("./smiles.pk", "rb") as a:
    smiles = pickle.load(a)

smiles

['Cc1ccc([C@@H]2C(C(=O)c3cc(C)ccc3C)=C(O)C(=O)N2CCCN2CCOCC2)cc1',
 'C[C@@H]1CC=C[C@@H]2[C@@H]3O[C@]3(C)[C@H](C)[C@@H]3[C@H](Cc4ccccc4)NC(=O)[C@]23C(=O)[C@@H]2O[C@@H]2[C@H](C)C1',
 'COc1ccccc1/C=C1\\SC(Nc2ccccc2)=C(C(=O)NC2CCN(C)CC2)C1=O',
 'C#CCOc1c(I)cc([C@H]2NC(=O)c3ccccc3O2)cc1OCC',
 'C=C(C)CN1C(=O)N[C@@](CC)(C2CCN(C(=O)c3cc(C)nc4ccc(C)cc34)CC2)C1=O',
 'CCN(CC)CCN1C(=O)C(O)=C(C(=O)/C=C/c2ccccc2)[C@H]1c1ccc([N+](=O)[O-])cc1',
 'N#Cc1ccc(-c2cccc(CNS(=O)(=O)c3cc(Br)cnc3N)c2)s1',
 'C=CCC1(CC=C)CCCN1C(=O)C[C@H]1C(=O)NCCN1Cc1cccc(C(F)(F)F)c1',
 'COC(=O)[C@]1(c2ccccc2)CCN(C(=O)Cc2csc(N=C(O)c3ccccc3)n2)C1',
 'CCN(CC)CCN1C(=O)C(O)=C(C(=O)/C=C/c2ccccc2)[C@@H]1c1ccc([N+](=O)[O-])cc1',
 'CC#CCN1C(=O)N[C@](CCc2ccccc2)(C2CCN(C(=O)C3CCCCC3)CC2)C1=O',
 'C=CCn1c(SCC(=O)NCc2ccccc2)nnc1[C@@H](C)NC(=O)c1ccccc1C',
 'CC#CCN1C(=O)N[C@@](Cc2cccc(F)c2)(C2CCN(C/C=C/c3ccco3)CC2)C1=O',
 'C[C@@H]1CC=C[C@@H]2[C@@H]3O[C@]3(C)[C@H](C)[C@@H]3[C@H](Cc4ccccc4)NC(=O)[C@]23C(=O)[C@@H]2O[C@H]2[C@H](C)C1',
 'C[C@H]1C[C@H

In [9]:
len(smiles)

2000

In [2]:
import pickle

In [3]:
with open("./output.pk", "rb") as f:
    data = pickle.load(f)

data

[tensor([[0.8597],
         [0.8711],
         [0.8811],
         [0.6916],
         [0.9249],
         [0.8524],
         [0.8367],
         [0.9591],
         [0.8607],
         [0.8524],
         [0.9755],
         [0.8550],
         [0.9356],
         [0.8711],
         [0.9021],
         [0.9371],
         [0.8499],
         [0.8499],
         [0.8866],
         [0.6916],
         [0.7547],
         [0.9609],
         [0.8905],
         [0.9090],
         [0.9299],
         [0.7547],
         [0.9112],
         [0.8866],
         [0.9043],
         [0.9249],
         [0.9637],
         [0.7844]]),
 tensor([[0.9016],
         [0.8711],
         [0.8550],
         [0.9258],
         [0.9005],
         [0.8491],
         [0.8553],
         [0.9605],
         [0.8774],
         [0.8959],
         [0.8905],
         [0.7844],
         [0.7990],
         [0.9258],
         [0.8711],
         [0.9755],
         [0.8885],
         [0.8985],
         [0.8885],
         [0.9605],
         [

In [4]:
with open("./smiles.pk", "rb") as a:
    smiles = pickle.load(a)

smiles

['Cc1ccc([C@@H]2C(C(=O)c3cc(C)ccc3C)=C(O)C(=O)N2CCCN2CCOCC2)cc1',
 'C[C@@H]1CC=C[C@@H]2[C@@H]3O[C@]3(C)[C@H](C)[C@@H]3[C@H](Cc4ccccc4)NC(=O)[C@]23C(=O)[C@@H]2O[C@@H]2[C@H](C)C1',
 'COc1ccccc1/C=C1\\SC(Nc2ccccc2)=C(C(=O)NC2CCN(C)CC2)C1=O',
 'C#CCOc1c(I)cc([C@H]2NC(=O)c3ccccc3O2)cc1OCC',
 'C=C(C)CN1C(=O)N[C@@](CC)(C2CCN(C(=O)c3cc(C)nc4ccc(C)cc34)CC2)C1=O',
 'CCN(CC)CCN1C(=O)C(O)=C(C(=O)/C=C/c2ccccc2)[C@H]1c1ccc([N+](=O)[O-])cc1',
 'N#Cc1ccc(-c2cccc(CNS(=O)(=O)c3cc(Br)cnc3N)c2)s1',
 'C=CCC1(CC=C)CCCN1C(=O)C[C@H]1C(=O)NCCN1Cc1cccc(C(F)(F)F)c1',
 'COC(=O)[C@]1(c2ccccc2)CCN(C(=O)Cc2csc(N=C(O)c3ccccc3)n2)C1',
 'CCN(CC)CCN1C(=O)C(O)=C(C(=O)/C=C/c2ccccc2)[C@@H]1c1ccc([N+](=O)[O-])cc1',
 'CC#CCN1C(=O)N[C@](CCc2ccccc2)(C2CCN(C(=O)C3CCCCC3)CC2)C1=O',
 'C=CCn1c(SCC(=O)NCc2ccccc2)nnc1[C@@H](C)NC(=O)c1ccccc1C',
 'CC#CCN1C(=O)N[C@@](Cc2cccc(F)c2)(C2CCN(C/C=C/c3ccco3)CC2)C1=O',
 'C[C@@H]1CC=C[C@@H]2[C@@H]3O[C@]3(C)[C@H](C)[C@@H]3[C@H](Cc4ccccc4)NC(=O)[C@]23C(=O)[C@@H]2O[C@H]2[C@H](C)C1',
 'C[C@H]1C[C@H

In [6]:
concat_data = {"smiles": smiles, "bbbp_score" : data}

In [7]:
concat_data

{'smiles': ['Cc1ccc([C@@H]2C(C(=O)c3cc(C)ccc3C)=C(O)C(=O)N2CCCN2CCOCC2)cc1',
  'C[C@@H]1CC=C[C@@H]2[C@@H]3O[C@]3(C)[C@H](C)[C@@H]3[C@H](Cc4ccccc4)NC(=O)[C@]23C(=O)[C@@H]2O[C@@H]2[C@H](C)C1',
  'COc1ccccc1/C=C1\\SC(Nc2ccccc2)=C(C(=O)NC2CCN(C)CC2)C1=O',
  'C#CCOc1c(I)cc([C@H]2NC(=O)c3ccccc3O2)cc1OCC',
  'C=C(C)CN1C(=O)N[C@@](CC)(C2CCN(C(=O)c3cc(C)nc4ccc(C)cc34)CC2)C1=O',
  'CCN(CC)CCN1C(=O)C(O)=C(C(=O)/C=C/c2ccccc2)[C@H]1c1ccc([N+](=O)[O-])cc1',
  'N#Cc1ccc(-c2cccc(CNS(=O)(=O)c3cc(Br)cnc3N)c2)s1',
  'C=CCC1(CC=C)CCCN1C(=O)C[C@H]1C(=O)NCCN1Cc1cccc(C(F)(F)F)c1',
  'COC(=O)[C@]1(c2ccccc2)CCN(C(=O)Cc2csc(N=C(O)c3ccccc3)n2)C1',
  'CCN(CC)CCN1C(=O)C(O)=C(C(=O)/C=C/c2ccccc2)[C@@H]1c1ccc([N+](=O)[O-])cc1',
  'CC#CCN1C(=O)N[C@](CCc2ccccc2)(C2CCN(C(=O)C3CCCCC3)CC2)C1=O',
  'C=CCn1c(SCC(=O)NCc2ccccc2)nnc1[C@@H](C)NC(=O)c1ccccc1C',
  'CC#CCN1C(=O)N[C@@](Cc2cccc(F)c2)(C2CCN(C/C=C/c3ccco3)CC2)C1=O',
  'C[C@@H]1CC=C[C@@H]2[C@@H]3O[C@]3(C)[C@H](C)[C@@H]3[C@H](Cc4ccccc4)NC(=O)[C@]23C(=O)[C@@H]2O[C@H]2[C@

## GLAM에 넣을 최종 데이터 만들기

In [3]:
import pandas as pd

In [5]:
data = pd.read_csv("./main_test/raw/merge_data_with_lipinski_tanimoto_and_common(only_smiles).csv", index_col=0)
data.head()

,smiles
0,Cc1ccc([C@@H]2C(C(=O)c3cc(C)ccc3C)=C(O)C(=O)N2...
1,C[C@@H]1CC=C[C@@H]2[C@@H]3O[C@]3(C)[C@H](C)[C@...
2,COc1ccccc1/C=C1\SC(Nc2ccccc2)=C(C(=O)NC2CCN(C)...
3,C#CCOc1c(I)cc([C@H]2NC(=O)c3ccccc3O2)cc1OCC
4,C=C(C)CN1C(=O)N[C@@](CC)(C2CCN(C(=O)c3cc(C)nc4...


In [6]:
data["BBBP"] = -1

In [7]:
data.head()

,smiles,BBBP
0,Cc1ccc([C@@H]2C(C(=O)c3cc(C)ccc3C)=C(O)C(=O)N2...,-1
1,C[C@@H]1CC=C[C@@H]2[C@@H]3O[C@]3(C)[C@H](C)[C@...,-1
2,COc1ccccc1/C=C1\SC(Nc2ccccc2)=C(C(=O)NC2CCN(C)...,-1
3,C#CCOc1c(I)cc([C@H]2NC(=O)c3ccccc3O2)cc1OCC,-1
4,C=C(C)CN1C(=O)N[C@@](CC)(C2CCN(C(=O)c3cc(C)nc4...,-1


In [8]:
data.to_csv("merge_data_with_lipinski_tanimoto_and_common(only_smiles).csv")

merge_data_with_lipinski_tanimoto_and_common(only_smiles) 이 파일을 tanimoto로 바꿔서 진행했음(실제로는 tanimoto, k-means 다 있는 데이터임)

# output_.pk 파일에서 필요한 결과만 파싱 (compounds 대상으로)

In [1]:
import pickle

In [2]:
with open("./output_.pk", "rb") as f:
    data = pickle.load(f)

len(data)

2196

In [1]:
import torch

In [2]:
data = torch.load("../../Dataset/GLAM-GP/processed/dataset_tanimoto.pt")
data

(Data(x=[2158631, 15], edge_index=[2, 4693754], edge_attr=[4693754, 4], y=[70244, 1], smi=[70244]),
 {'x': tensor([      0,      33,      66,  ..., 2158585, 2158608, 2158631]),
  'edge_index': tensor([      0,      72,     148,  ..., 4693654, 4693704, 4693754]),
  'edge_attr': tensor([      0,      72,     148,  ..., 4693654, 4693704, 4693754]),
  'y': tensor([    0,     1,     2,  ..., 70242, 70243, 70244]),
  'smi': tensor([    0,     1,     2,  ..., 70242, 70243, 70244])})

In [3]:
data[0]

Data(x=[2158631, 15], edge_index=[2, 4693754], edge_attr=[4693754, 4], y=[70244, 1], smi=[70244])

In [4]:
data[1]

{'x': tensor([      0,      33,      66,  ..., 2158585, 2158608, 2158631]),
 'edge_index': tensor([      0,      72,     148,  ..., 4693654, 4693704, 4693754]),
 'edge_attr': tensor([      0,      72,     148,  ..., 4693654, 4693704, 4693754]),
 'y': tensor([    0,     1,     2,  ..., 70242, 70243, 70244]),
 'smi': tensor([    0,     1,     2,  ..., 70242, 70243, 70244])}

In [1]:
import pickle

In [2]:
with open("./smiles.pk", "rb") as f:
    smi = pickle.load(f)

len(smi)

70244

In [3]:
with open("./output_.pk", "rb") as a:
    data = pickle.load(a)

len(data)

2196

In [4]:
import torch
import numpy as np

In [5]:
data

[tensor([[0.9591],
         [0.9042],
         [0.9116],
         [0.5298],
         [0.8570],
         [0.4498],
         [0.5367],
         [0.9595],
         [0.5721],
         [0.4498],
         [0.9256],
         [0.5996],
         [0.8763],
         [0.9042],
         [0.5794],
         [0.9580],
         [0.4498],
         [0.4498],
         [0.5253],
         [0.5298],
         [0.5704],
         [0.7962],
         [0.9548],
         [0.9600],
         [0.9389],
         [0.5704],
         [0.7428],
         [0.5253],
         [0.4447],
         [0.8570],
         [0.9581],
         [0.8190]]),
 tensor([[0.3362],
         [0.8901],
         [0.5996],
         [0.9412],
         [0.3356],
         [0.6822],
         [0.4492],
         [0.9595],
         [0.7360],
         [0.3051],
         [0.9548],
         [0.8190],
         [0.8798],
         [0.9412],
         [0.8901],
         [0.9256],
         [0.7724],
         [0.7917],
         [0.7724],
         [0.9595],
         [

In [6]:
type(data)

list

In [7]:
data[0]

tensor([[0.9591],
        [0.9042],
        [0.9116],
        [0.5298],
        [0.8570],
        [0.4498],
        [0.5367],
        [0.9595],
        [0.5721],
        [0.4498],
        [0.9256],
        [0.5996],
        [0.8763],
        [0.9042],
        [0.5794],
        [0.9580],
        [0.4498],
        [0.4498],
        [0.5253],
        [0.5298],
        [0.5704],
        [0.7962],
        [0.9548],
        [0.9600],
        [0.9389],
        [0.5704],
        [0.7428],
        [0.5253],
        [0.4447],
        [0.8570],
        [0.9581],
        [0.8190]])

In [8]:
type(data[0])

torch.Tensor

In [9]:
prac = data[0].tolist()

In [10]:
prac[0]

[0.9591167569160461]

In [11]:
prac[0][0]

0.9591167569160461

In [13]:
lst = []
for i in range(len(data)):
    data[i] = data[i].tolist()
    for x in range(len(data[i])):
        lst.append(data[i][x])

lst

[[0.9591167569160461],
 [0.9041711688041687],
 [0.9116113185882568],
 [0.5298000574111938],
 [0.8570442199707031],
 [0.4497854709625244],
 [0.5366774797439575],
 [0.9594985246658325],
 [0.5721033811569214],
 [0.4497852325439453],
 [0.9256282448768616],
 [0.5996081829071045],
 [0.8762694597244263],
 [0.9041711688041687],
 [0.5793989896774292],
 [0.9579871892929077],
 [0.4497854709625244],
 [0.4497855603694916],
 [0.5253028273582458],
 [0.5298002362251282],
 [0.5703928470611572],
 [0.7961773872375488],
 [0.9548159241676331],
 [0.960026204586029],
 [0.9388957023620605],
 [0.5703930258750916],
 [0.7427502274513245],
 [0.5253030061721802],
 [0.4447474777698517],
 [0.8570442795753479],
 [0.958121657371521],
 [0.8189746737480164],
 [0.33618980646133423],
 [0.8900923132896423],
 [0.5996082425117493],
 [0.9411923289299011],
 [0.33560991287231445],
 [0.6822324991226196],
 [0.4492126703262329],
 [0.959527313709259],
 [0.7360358834266663],
 [0.30506646633148193],
 [0.9548159241676331],
 [0.8189746

In [14]:
finally_lst = []
for i in range(len(lst)):
    finally_lst.append(lst[i][0])

finally_lst

[0.9591167569160461,
 0.9041711688041687,
 0.9116113185882568,
 0.5298000574111938,
 0.8570442199707031,
 0.4497854709625244,
 0.5366774797439575,
 0.9594985246658325,
 0.5721033811569214,
 0.4497852325439453,
 0.9256282448768616,
 0.5996081829071045,
 0.8762694597244263,
 0.9041711688041687,
 0.5793989896774292,
 0.9579871892929077,
 0.4497854709625244,
 0.4497855603694916,
 0.5253028273582458,
 0.5298002362251282,
 0.5703928470611572,
 0.7961773872375488,
 0.9548159241676331,
 0.960026204586029,
 0.9388957023620605,
 0.5703930258750916,
 0.7427502274513245,
 0.5253030061721802,
 0.4447474777698517,
 0.8570442795753479,
 0.958121657371521,
 0.8189746737480164,
 0.33618980646133423,
 0.8900923132896423,
 0.5996082425117493,
 0.9411923289299011,
 0.33560991287231445,
 0.6822324991226196,
 0.4492126703262329,
 0.959527313709259,
 0.7360358834266663,
 0.30506646633148193,
 0.9548159241676331,
 0.8189746141433716,
 0.8797939419746399,
 0.9411923885345459,
 0.8900923132896423,
 0.9256282448

In [15]:
len(finally_lst)

70244

In [16]:
len(smi)

70244

In [17]:
import pandas as pd

In [18]:
smi = pd.DataFrame(smi)

In [19]:
finally_lst = pd.DataFrame(finally_lst)

In [20]:
final_data = pd.concat([smi, finally_lst], axis=1)

In [21]:
final_data.head()

,0,0
0,Cc1ccc([C@@H]2C(C(=O)c3cc(C)ccc3C)=C(O)C(=O)N2...,0.959117
1,C[C@@H]1CC=C[C@@H]2[C@@H]3O[C@]3(C)[C@H](C)[C@...,0.904171
2,COc1ccccc1/C=C1\SC(Nc2ccccc2)=C(C(=O)NC2CCN(C)...,0.911611
3,C#CCOc1c(I)cc([C@H]2NC(=O)c3ccccc3O2)cc1OCC,0.529800
4,C=C(C)CN1C(=O)N[C@@](CC)(C2CCN(C(=O)c3cc(C)nc4...,0.857044


In [22]:
final_data.columns = ["smiles", "BBBP_Score"]

In [23]:
final_data.head()

,smiles,BBBP_Score
0,Cc1ccc([C@@H]2C(C(=O)c3cc(C)ccc3C)=C(O)C(=O)N2...,0.959117
1,C[C@@H]1CC=C[C@@H]2[C@@H]3O[C@]3(C)[C@H](C)[C@...,0.904171
2,COc1ccccc1/C=C1\SC(Nc2ccccc2)=C(C(=O)NC2CCN(C)...,0.911611
3,C#CCOc1c(I)cc([C@H]2NC(=O)c3ccccc3O2)cc1OCC,0.529800
4,C=C(C)CN1C(=O)N[C@@](CC)(C2CCN(C(=O)c3cc(C)nc4...,0.857044


In [24]:
len(final_data)

70244

In [26]:
final_data.isnull().sum()

smiles        0
BBBP_Score    0
dtype: int64

In [27]:
final_data.to_csv("GLAM_results.csv", index=False)

# output_.pk 로부터 필요한 결과만 파싱(NZ27만)

In [1]:
import pickle

In [2]:
with open("./output_.pk", "rb") as f:
    data = pickle.load(f)

len(data)

2196

In [3]:
import torch

In [4]:
data[0]

tensor([[0.8932],
        [0.5876],
        [0.9939],
        [0.9324],
        [0.7587],
        [0.8954],
        [0.2146],
        [0.9924],
        [0.7845],
        [0.8954],
        [0.9313],
        [0.1136],
        [0.9427],
        [0.5876],
        [0.2395],
        [0.9662],
        [0.8996],
        [0.8996],
        [0.7302],
        [0.9324],
        [0.1437],
        [0.9215],
        [0.9991],
        [0.8766],
        [0.9964],
        [0.1437],
        [0.2122],
        [0.7302],
        [0.3779],
        [0.7587],
        [0.8725],
        [0.4275]])

In [5]:
data[1]

tensor([[0.5735],
        [0.5876],
        [0.1136],
        [0.9851],
        [0.5581],
        [0.9894],
        [0.8933],
        [0.9953],
        [0.9499],
        [0.4584],
        [0.9991],
        [0.4275],
        [0.4383],
        [0.9851],
        [0.5876],
        [0.9313],
        [0.5289],
        [0.6629],
        [0.5289],
        [0.9953],
        [0.2395],
        [0.1610],
        [0.2395],
        [0.8766],
        [0.9996],
        [0.5375],
        [0.9427],
        [0.7845],
        [0.4435],
        [0.1489],
        [0.9215],
        [0.8932]])

In [6]:
with open("./smiles.pk", "rb") as f:
    smi = pickle.load(f)

len(smi)

70245

In [7]:
import numpy as np

In [8]:
type(data)

list

In [9]:
type(data[0])

torch.Tensor

In [10]:
prac = data[0].tolist()

In [11]:
prac[0]

[0.8931832909584045]

In [13]:
prac[0][0]

0.8931832909584045

In [14]:
lst = []
for i in range(len(data)):
    data[i] = data[i].tolist()
    for x in range(len(data[i])):
        lst.append(data[i][x])

lst

[[0.8931832909584045],
 [0.5875986218452454],
 [0.9938555359840393],
 [0.9323638677597046],
 [0.7587229609489441],
 [0.8953829407691956],
 [0.21460431814193726],
 [0.9923763871192932],
 [0.784548819065094],
 [0.8953830003738403],
 [0.9313293099403381],
 [0.11358156055212021],
 [0.9426687955856323],
 [0.5875987410545349],
 [0.2395235151052475],
 [0.9661557674407959],
 [0.8996272087097168],
 [0.8996272087097168],
 [0.7301650643348694],
 [0.9323638677597046],
 [0.14367452263832092],
 [0.9214720726013184],
 [0.9990569949150085],
 [0.8765978813171387],
 [0.9963937401771545],
 [0.14367438852787018],
 [0.21217042207717896],
 [0.7301648855209351],
 [0.37794673442840576],
 [0.7587229013442993],
 [0.872477650642395],
 [0.42752647399902344],
 [0.5735034346580505],
 [0.5875985622406006],
 [0.11358168721199036],
 [0.985081672668457],
 [0.5581217408180237],
 [0.9893695116043091],
 [0.8932978510856628],
 [0.9953298568725586],
 [0.9499086141586304],
 [0.45839518308639526],
 [0.9990569949150085],
 [0.4

In [16]:
finally_lst = []
for i in range(len(lst)):
    finally_lst.append(lst[i][0])

finally_lst

[0.8931832909584045,
 0.5875986218452454,
 0.9938555359840393,
 0.9323638677597046,
 0.7587229609489441,
 0.8953829407691956,
 0.21460431814193726,
 0.9923763871192932,
 0.784548819065094,
 0.8953830003738403,
 0.9313293099403381,
 0.11358156055212021,
 0.9426687955856323,
 0.5875987410545349,
 0.2395235151052475,
 0.9661557674407959,
 0.8996272087097168,
 0.8996272087097168,
 0.7301650643348694,
 0.9323638677597046,
 0.14367452263832092,
 0.9214720726013184,
 0.9990569949150085,
 0.8765978813171387,
 0.9963937401771545,
 0.14367438852787018,
 0.21217042207717896,
 0.7301648855209351,
 0.37794673442840576,
 0.7587229013442993,
 0.872477650642395,
 0.42752647399902344,
 0.5735034346580505,
 0.5875985622406006,
 0.11358168721199036,
 0.985081672668457,
 0.5581217408180237,
 0.9893695116043091,
 0.8932978510856628,
 0.9953298568725586,
 0.9499086141586304,
 0.45839518308639526,
 0.9990569949150085,
 0.42752665281295776,
 0.43826547265052795,
 0.9850817918777466,
 0.5875986218452454,
 0.93

In [17]:
len(finally_lst)

70245

In [18]:
len(smi)

70245

In [19]:
import pandas as pd

In [20]:
smi = pd.DataFrame(smi)

In [21]:
finally_lst = pd.DataFrame(finally_lst)

In [22]:
final_data = pd.concat([smi, finally_lst], axis=1)

In [23]:
final_data.head()

,0,0
0,Cc1ccc([C@@H]2C(C(=O)c3cc(C)ccc3C)=C(O)C(=O)N2...,0.893183
1,C[C@@H]1CC=C[C@@H]2[C@@H]3O[C@]3(C)[C@H](C)[C@...,0.587599
2,COc1ccccc1/C=C1\SC(Nc2ccccc2)=C(C(=O)NC2CCN(C)...,0.993856
3,C#CCOc1c(I)cc([C@H]2NC(=O)c3ccccc3O2)cc1OCC,0.932364
4,C=C(C)CN1C(=O)N[C@@](CC)(C2CCN(C(=O)c3cc(C)nc4...,0.758723


In [24]:
final_data.columns = ["smiles", "BBBP_Score"]

In [25]:
final_data.head()

,smiles,BBBP_Score
0,Cc1ccc([C@@H]2C(C(=O)c3cc(C)ccc3C)=C(O)C(=O)N2...,0.893183
1,C[C@@H]1CC=C[C@@H]2[C@@H]3O[C@]3(C)[C@H](C)[C@...,0.587599
2,COc1ccccc1/C=C1\SC(Nc2ccccc2)=C(C(=O)NC2CCN(C)...,0.993856
3,C#CCOc1c(I)cc([C@H]2NC(=O)c3ccccc3O2)cc1OCC,0.932364
4,C=C(C)CN1C(=O)N[C@@](CC)(C2CCN(C(=O)c3cc(C)nc4...,0.758723


In [26]:
len(final_data)

70245

In [27]:
final_data.isnull().sum()

smiles        0
BBBP_Score    0
dtype: int64

In [29]:
ZN27 = final_data.iloc[70244]

In [31]:
ZN27

smiles        COc1ccc([C@@H]2CCCN2C(=O)Nc2cc(C(F)(F)F)ccc2N2...
BBBP_Score                                             0.976869
Name: 70244, dtype: object

In [34]:
ZN27 = pd.DataFrame(ZN27).T
ZN27.head()

,smiles,BBBP_Score
70244,COc1ccc([C@@H]2CCCN2C(=O)Nc2cc(C(F)(F)F)ccc2N2...,0.976869


In [35]:
ZN27.to_csv("ZN27_BBBP_result.csv", index=False)